In [8]:
search_for = 86
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004458904266357422
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 108506231
type: [1, 1, 1, 1, 86] 86
cases of this type: 54700816
10000 19.679953220125537
20000 18.953256050331174
30000 63.32250877012189
40000 75.24754235614897
50000 59.6427579393415
60000 61.85226812631184
70000 65.94712105969104
80000 63.25759277995712
90000 61.570000711287115
100000 62.56414202495501
110000 59.38792106710614
120000 59.532470362429684
130000 61.27881759366251
140000 62.61144927674734
150000 60.20079307610322
160000 64.37662549893992
170000 68.82805171706575
180000 60.29104359315623
190000 59.994094613958545
200000 59.54454852800634
210000 60.07303429578376
220000 56.222098596364646
230000 59.381242715436976
240000 59.65100913456119
250000 58.79823274912943
260000 58.69049950574088
270000 60.80901017880787
280000 59.110965208017475
290000 55.98274568893842
300000 64.04675405785038
310000 31.895385097662768
3

3100000 49.05988402160853
3110000 51.915483192256225
3120000 51.254077655127396
3130000 50.53080897914992
3140000 44.49997928335672
3150000 22.06201113530449
3160000 21.506546210151118
3170000 21.525228029577377
3180000 59.95710217603366
3190000 55.838986999944716
3200000 58.33916798430453
3210000 29.41092988343878
3220000 17.848911965866066
3230000 17.843867260702133
3240000 20.010462961604265
3250000 62.65202850157977
3260000 51.70153958289486
3270000 51.0930060638646
3280000 53.8499233141377
3290000 49.74214541184373
3300000 47.7841478584248
3310000 50.793932761461015
3320000 51.438311738982804
3330000 53.922030897282376
3340000 56.47260984766825
3350000 52.33721400773611
3360000 54.75008373406815
3370000 57.35338278471659
3380000 52.06423096891996
3390000 50.209158280275766
3400000 49.7837805521591
3410000 50.421120677941204
3420000 50.171864122636244
3430000 52.85118634763847
3440000 54.1654102351823
3450000 53.20591855765815
3460000 49.84773355202569
3470000 45.925036937555575
34

6230000 49.63998609710155
6240000 56.929250248256
6250000 52.64132543168709
6260000 48.28636473292299
6270000 53.76022920986883
6280000 52.35747922113127
6290000 21.364066699765026
6300000 20.83326570720521
6310000 21.09727876733801
6320000 41.63737700272104
6330000 57.60273825539795
6340000 46.09145273322956
6350000 53.693524362135015
6360000 57.86830107549359
6370000 55.70054167318075
6380000 56.09276204120939
6390000 54.986805202445915
6400000 49.2526268614203
6410000 50.88841542019184
6420000 60.784584883482545
6430000 17.38887070122376
6440000 17.660657061021844
6450000 17.39387108201405
6460000 36.939305759101934
6470000 64.8549762144677
6480000 47.225033081132
6490000 51.41435908548459
6500000 57.90486120292131
6510000 49.423096405346904
6520000 46.98993949366013
6530000 50.6635372502649
6540000 57.39561507997958
6550000 48.64139355313498
6560000 50.28430454144609
6570000 56.01288301353698
6580000 45.572913920252276
6590000 47.22535175012344
6600000 50.30569500947638
6610000 43.

9360000 44.83629561034714
9370000 45.466740526219176
9380000 46.77480935931689
9390000 46.03042052498537
9400000 46.61697802998225
9410000 46.57579646948284
9420000 49.89897428750751
9430000 29.856566192929417
9440000 20.72784918311775
9450000 20.65271323665696
9460000 24.756466209683115
9470000 53.78148072604275
9480000 46.45109567839749
9490000 47.12056078473049
9500000 49.06725842971182
9510000 46.0247249571426
9520000 44.15530879301613
9530000 48.22951847878235
9540000 51.0460110916144
9550000 51.289948760974774
9560000 49.806549252058154
9570000 49.23674504912362
9580000 45.683678046636246
9590000 45.880584095764455
9600000 49.75772283035848
9610000 48.83955583171303
9620000 45.828608428704854
9630000 50.69556728825265
9640000 45.51935291423724
9650000 16.881608785551787
9660000 17.28930315906374
9670000 17.05494402615585
9680000 38.435653125478964
9690000 51.093542959536045
9700000 43.85971711197993
9710000 46.840483324134226
9720000 48.280112647365165
9730000 44.04856007240796
9

12390000 44.105007084791914
12400000 8.639217274685132
12410000 8.572228659858228
12420000 8.43601912511384
12430000 28.890929054089764
12440000 49.45127741816183
12450000 40.23160758204409
12460000 40.04281284169139
12470000 43.599724077652
12480000 41.22311310093674
12490000 41.920775765148285
12500000 43.849753871921116
12510000 42.044030268800775
12520000 39.03360563606792
12530000 44.03063065078984
12540000 41.133208699668636
12550000 41.67591355606294
12560000 46.56330045839576
12570000 34.04456131971146
12580000 19.73545187119141
12590000 19.523052704577147
12600000 19.704844555260298
12610000 44.22152097589038
12620000 45.176670815114896
12630000 41.98646076814575
12640000 49.62159095187002
12650000 47.40741209674072
12660000 44.40874181387512
12670000 46.24613172937383
12680000 46.22142634297311
12690000 46.87126540338411
12700000 50.1137812537035
12710000 49.16579013738356
12720000 50.01111196538102
12730000 56.208140370385735
12740000 49.437599981930674
12750000 50.145044514

...

96260000 3.059764955293874
96270000 3.393017474430435
96280000 3.397858424624052
96290000 3.151606973601401
96300000 3.3164242899390657
96310000 3.2812386314133932
96320000 2.8831538055860997
96330000 1.6377454887160061
96340000 1.7337570078912512
96350000 4.1427449904020825
96360000 3.539310141638809
96370000 3.552644932207631
96380000 2.9811035514924527
96390000 2.1324398737359775
96400000 1.9547351033113398
96410000 2.645450586168667
96420000 3.60852792338454
96430000 3.065361329508172
96440000 3.5965901079608398
96450000 3.245290429806219
96460000 2.7790849912280375
96470000 3.1552844397810103
96480000 3.3413608336560925
96490000 2.812995746594515
96500000 2.8472841247114338
96510000 3.6580119682340095
96520000 0.9723742712272869
96530000 0.9447114195148347
96540000 0.8216777489434084
96550000 3.0698862407633136
96560000 3.067909243734956
96570000 3.260300869772957
96580000 3.416393846123775
96590000 3.1601737955162523
96600000 3.1531065738574005
96610000 2.8595151906439664
9662000

99120000 2.7563819013612667
99130000 2.5094204477184947
99140000 2.267544741072039
99150000 1.1073286133722662
99160000 2.299085226971381
99170000 2.3192104197138548
99180000 2.2345368702329136
99190000 1.735749013575044
99200000 0.8209098469085894
99210000 1.5335995413400796
99220000 2.23715759778183
99230000 2.477920368102332
99240000 2.525325990377817
99250000 1.631421203679045
99260000 1.8379754753081203
99270000 2.5073749556401568
99280000 2.6258791928622918
99290000 2.30755618973734
99300000 2.4840358091082773
99310000 1.3955723467304308
99320000 1.7563733409424225
99330000 2.3629844304963945
99340000 2.03248121751778
99350000 1.9575243252614143
99360000 0.8132851368278132
99370000 1.6705003292169969
99380000 2.2473883596588373
99390000 2.479743745377064
99400000 1.7412306391314734
99410000 1.4496695137224396
99420000 2.378159296614859
99430000 2.5175035489124458
99440000 2.2387061363365053
99450000 2.4863885284203757
99460000 2.486774087386655
99470000 1.6477211130190095
9948000

102010000 1.7257232973368697
102020000 1.7613336206240853
102030000 1.8782230616704623
102040000 1.6575632923320864
102050000 1.1837737280107141
102060000 0.540063849613693
102070000 0.6279980762176977
102080000 1.0012303559101026
102090000 1.7106352554246718
102100000 1.9352460345928073
102110000 1.748452488099118
102120000 1.8112100944513014
102130000 1.8007183164174425
102140000 1.9862004274587035
102150000 1.7847657941073376
102160000 1.749685809096747
102170000 1.6446436723127165
102180000 1.7740443818029497
102190000 1.7804683113250401
102200000 1.2223323696427544
102210000 0.9985471634008288
102220000 0.53571946979479
102230000 0.594743218971312
102240000 1.0127383835680948
102250000 1.3890018822348118
102260000 1.7375581923407317
102270000 1.7833898417996301
102280000 1.5465152453390822
102290000 1.6913105922810039
102300000 1.7179735011283557
102310000 1.8029109383737578
102320000 1.7229536682982445
102330000 1.7173611504557
102340000 1.724304085750434
102350000 1.467372399812

104850000 0.32820917540540295
104860000 0.25165313112012544
104870000 0.25347194488759833
104880000 0.2376131881961359
104890000 0.24277043368443518
104900000 0.2157835255341331
104910000 0.24377229795639382
104920000 0.2473781612469024
104930000 0.24282137885820868
104940000 0.2429122513424953
104950000 0.23993370360300273
104960000 0.240082123657763
104970000 0.23577701007688712
104980000 0.22153842011229194
104990000 0.2407701733864546
105000000 0.24222133429315354
best so far: 0
type: [1, 2, 43, 1, 1] 86
cases of this type: 3698
best so far: 0
type: [1, 43, 1, 1, 2] 86
cases of this type: 688
best so far: 0
type: [1, 43, 1, 2, 1] 86
cases of this type: 344
best so far: 0
type: [1, 43, 2, 1, 1] 86
cases of this type: 172
best so far: 0
type: [1, 86, 1, 1, 1] 86
cases of this type: 86
best so far: 0
type: [2, 1, 1, 1, 43] 86
cases of this type: 3418801
105010000 0.37870099043944144
105020000 1.0379741030664444
105030000 0.9904069540272025
105040000 1.0802459717469746
105050000 0.3875

107520000 0.18452700081845785
107530000 0.20076077867359585
107540000 0.22489767401987315
107550000 0.26539910411101575
107560000 0.2626506865827243
107570000 0.20414165057261785
107580000 0.21670416326292358
107590000 0.1705567293779916
107600000 0.16950388098756472
107610000 0.1856342985674474
107620000 0.1889682898422347
107630000 0.24111467352533342
107640000 0.24198438578220208
107650000 0.1832922549792197
107660000 0.22245838848102095
107670000 0.14334012043225766
107680000 0.166359581910842
107690000 0.1813725146883329
107700000 0.1651302395061718
107710000 0.2110418285702467
107720000 0.219053275059402
107730000 0.13245350161976285
107740000 0.20725434508222343
107750000 0.12103156228264172
107760000 0.1463621143394245
107770000 0.17735509795471693
107780000 0.13295133330835898
107790000 0.20148095655996934
107800000 0.17492817731391722
107810000 0.1468861303343177
107820000 0.1801260172190666
107830000 0.10944445521720914
107840000 0.13862931582564114
107850000 0.1718926499468